# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-02-24 18:40:06] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-02-24 18:40:06] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-02-24 18:40:06] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-02-24 18:40:09] INFO server_args.py:1828: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=96 avail_mem=14.76 GB):  20%|██        | 4/20 [00:00<00:01, 10.82it/s] 

Capturing batches (bs=48 avail_mem=14.11 GB):  45%|████▌     | 9/20 [00:00<00:00, 17.96it/s]

Capturing batches (bs=16 avail_mem=14.05 GB):  60%|██████    | 12/20 [00:00<00:00, 21.37it/s]

Capturing batches (bs=1 avail_mem=14.00 GB):  90%|█████████ | 18/20 [00:01<00:00, 19.56it/s]

Capturing batches (bs=1 avail_mem=14.00 GB): 100%|██████████| 20/20 [00:01<00:00, 17.75it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Lydia and I'm from New York City. I have a PhD in psychology and a master's in business administration. I have been employed at the Centre for Brain and Cognitive Sciences at the City University of New York (CUNY) since 2002. I have worked on research in several areas including children’s brain development, sensory and motor skills, language development, and cognitive development. My work has been featured in several publications, including in the journal Developmental Cognitive Neuroscience and in the journal Neuron and in the International Journal of Developmental Disorders.
I am a member of the American Psychological Association (APA), the National Association
Prompt: The president of the United States is
Generated text:  a position that has been in place since the founding of the United States. The presidency is filled with considerable power and responsibility, which is a place of great importance for the United States. A presidential ele

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [occupation] who has been working in [industry] for [number] years. I am passionate about [reason for passion], and I am always looking for ways to [action or achievement]. I am [age] years old, and I have [number] years of experience in [industry]. I am [gender] and I am [race]. I am [occupation] and I am [gender] and I am [race]. I am [occupation] and I am [gender] and I am [race]. I am [occupation] and I am [gender] and I am [race

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

A. True
B. False
A. True

Paris is the capital city of France, and it is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum. The city is also famous for its rich history, including the French Revolution and the French Revolution Monument. Paris is a major cultural and economic center in Europe, and it is home to many famous museums, theaters, and restaurants. The city is also known for its fashion industry, with many famous fashion designers and boutiques located in the city. Overall, Paris is a major city with a rich history

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to become more integrated with human intelligence, allowing it to learn from and adapt to human behavior and decision-making processes.

2. Enhanced ethical considerations: As AI becomes more advanced, there will be increased scrutiny of its ethical implications, including issues such as bias, transparency, and accountability.

3. Development of more advanced AI: As AI technology continues to advance, there will be a need for more advanced AI systems that can handle complex and unpredictable situations.

4. Increased focus on privacy and security: As AI becomes more



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Jane, a smart, curious, and adventurous young woman. I am a successful writer, with a passion for exploring new places and learning about different cultures. I am also a natural leader, and I thrive on pushing boundaries and trying new things. I believe in the power of curiosity and wonder to inspire and motivate others. Whether you're a student, a parent, or an employee, I'm here to help you find your voice and your purpose. How can I contribute to your life and help you reach your goals? Let's talk! 😊❤️✨🌈

Sure, here is a more detailed self-introduction for a fictional

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower and the Louvre Museum. Its rich history and vibrant culture make it a popular destination for tourists and locals alike. P

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 and

 learning

 styles

.

 This

In [6]:
llm.shutdown()